## Instructions

You are asked to complete the following files:
* **pruned_layers.py**, which contains the pruning of DNNs to reduce the storage of insignificant weight parameters with 2 methods: pruning by percentage and prune by standara deviation.
* **train_util.py**, which includes the training process of DNNs with pruned connections.
* **quantize.py**, which applies the quantization (weight sharing) part on the DNN to reduce the storage of weight parameters.
* **huffman_coding.py**, which applies the Huffman coding onto the weight of DNNs to further compress the weight size.

You are asked to submit the following files:
* **net_before_pruning.pt**, which is the weight parameters before applying pruning on DNN weight parameters.
* **net_after_pruning.pt**, which is the weight paramters after applying pruning on DNN weight parameters.
* **net_after_quantization.pt**, which is the weight parameters after applying quantization (weight sharing) on DNN weight parameters.
* **codebook_vgg16.npy**, which is the quantization codebook of each layer after applying quantization (weight sharing).
* **huffman_encoding.npy**, which is the encoding map of each item within the quantization codebook in the whole DNN architecture.
* **huffman_freq.npy**, which is the frequency map of each item within the quantization codebook in the whole DNN. 

To ensure fair grading policy, we fix the choice of model to VGG16_half, which is a down-scaled version of VGG16 using a width multiplier of 0.5. You may check the implementation in **vgg16.py** for more details.

In [1]:
from vgg16 import VGG16, VGG16_half
from train_util import train, finetune_after_prune, test, finetune_after_quantize
from quantize import quantize_whole_model
from huffman_coding import huffman_coding
from summary import summary
import torch
import numpy as np
from prune import prune

device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Full-precision model training

In [2]:
net = VGG16_half()
net = net.to(device)

# Uncomment to load pretrained weights
net.load_state_dict(torch.load("net_before_pruning.pt"))

# Comment if you have loaded pretrained weights
# Tune the hyperparameters here.
#train(net, epochs=100, batch_size=233, lr=0.0173, reg=1e-2)

<All keys matched successfully>

In [3]:
# Load the best weight paramters
net.load_state_dict(torch.load("net_before_pruning.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3027, Test accuracy=0.9177


In [4]:
print("-----Summary before pruning-----")
summary(net)
print("-------------------------------")

-----Summary before pruning-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		864			0.000000
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	9216		9216			0.000000
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	18432		18432			0.000000
8		BatchNorm	N/A		N/A			N/A
9		ReLU		N/A		N/A			N/A
10		Convolutional	36864		36864			0.000000
11		BatchNorm	N/A		N/A			N/A
12		ReLU		N/A		N/A			N/A
13		Convolutional	73728		73728			0.000000
14		BatchNorm	N/A		N/A			N/A
15		ReLU		N/A		N/A			N/A
16		Convolutional	147456		147456			0.000000
17		BatchNorm	N/A		N/A			N/A
18		ReLU		N/A		N/A			N/A
19		Convolutional	147456		147456			0.000000
20		BatchNorm	N/A		N/A			N/A
21		ReLU		N/A		N/A			N/A
22		Convolutional	294912		294912			0.000000
23		BatchNorm	N/A		N/A			N/A
24		ReLU		N/A		N/A			N/A
25		Convolutional	589824		589824			0.000000
26		BatchNorm	N/A		N/A			N/A
27		ReLU		N/A		N/A			N/A
28		Convolutional	589824		589824			0.000000
29		Batch

### Pruning & Finetune with pruned connections

In [5]:
# Test accuracy before fine-tuning
prune(net, method='percentage', q=45.0, s=0.75)
test(net)

Files already downloaded and verified
Test Loss=0.3275, Test accuracy=0.9082


In [6]:
# Uncomment to load pretrained weights
#net.load_state_dict(torch.load("net_after_pruning.pt"))
# Comment if you have loaded pretrained weights
finetune_after_prune(net, epochs=100, batch_size=128, lr=0.001, reg=5e-5)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=16]	Loss=0.1645	acc=0.9595	2269.7 examples/second
[Step=32]	Loss=0.1909	acc=0.9524	9808.8 examples/second
[Step=48]	Loss=0.2180	acc=0.9422	8862.8 examples/second
[Step=64]	Loss=0.2269	acc=0.9393	9667.4 examples/second
[Step=80]	Loss=0.2303	acc=0.9375	8606.5 examples/second
[Step=96]	Loss=0.2353	acc=0.9355	9302.9 examples/second
[Step=112]	Loss=0.2373	acc=0.9342	8623.9 examples/second
[Step=128]	Loss=0.2428	acc=0.9326	9782.2 examples/second
[Step=144]	Loss=0.2450	acc=0.9310	9850.2 examples/second
[Step=160]	Loss=0.2456	acc=0.9303	8889.9 examples/second
[Step=176]	Loss=0.2455	acc=0.9302	8926.7 examples/second
[Step=192]	Loss=0.2453	acc=0.9299	8168.4 examples/second
[Step=208]	Loss=0.2438	acc=0.9305	8587.8 examples/second
[Step=224]	Loss=0.2434	acc=0.9307	8087.0 examples/second
[Step=240]	Loss=0.2420	acc=0.9312	9218.5 examples/second
[Step=256]	Loss=0.2408	acc=0.9315	8484.2 exa

[Step=2208]	Loss=0.1883	acc=0.9405	9260.3 examples/second
[Step=2224]	Loss=0.1896	acc=0.9399	9518.8 examples/second
[Step=2240]	Loss=0.1906	acc=0.9396	8542.5 examples/second
[Step=2256]	Loss=0.1906	acc=0.9398	8756.6 examples/second
[Step=2272]	Loss=0.1908	acc=0.9398	8756.4 examples/second
[Step=2288]	Loss=0.1902	acc=0.9401	8513.4 examples/second
[Step=2304]	Loss=0.1896	acc=0.9403	10113.8 examples/second
[Step=2320]	Loss=0.1892	acc=0.9403	9278.1 examples/second
[Step=2336]	Loss=0.1897	acc=0.9401	7985.5 examples/second
Test Loss=0.3638, Test acc=0.8885

Epoch: 6
[Step=2352]	Loss=0.2108	acc=0.9271	1019.0 examples/second
[Step=2368]	Loss=0.1668	acc=0.9482	9253.8 examples/second
[Step=2384]	Loss=0.1673	acc=0.9470	8724.6 examples/second
[Step=2400]	Loss=0.1669	acc=0.9491	8946.2 examples/second
[Step=2416]	Loss=0.1746	acc=0.9470	8689.1 examples/second
[Step=2432]	Loss=0.1703	acc=0.9478	9109.6 examples/second
[Step=2448]	Loss=0.1765	acc=0.9462	7889.6 examples/second
[Step=2464]	Loss=0.1778	acc

[Step=4400]	Loss=0.1425	acc=0.9554	8288.4 examples/second
[Step=4416]	Loss=0.1418	acc=0.9558	8584.2 examples/second
[Step=4432]	Loss=0.1431	acc=0.9555	8846.2 examples/second
[Step=4448]	Loss=0.1443	acc=0.9549	9006.3 examples/second
[Step=4464]	Loss=0.1445	acc=0.9549	8150.5 examples/second
[Step=4480]	Loss=0.1431	acc=0.9557	9148.3 examples/second
[Step=4496]	Loss=0.1432	acc=0.9554	9983.3 examples/second
[Step=4512]	Loss=0.1457	acc=0.9547	8994.7 examples/second
[Step=4528]	Loss=0.1448	acc=0.9548	9010.3 examples/second
[Step=4544]	Loss=0.1468	acc=0.9543	7867.5 examples/second
[Step=4560]	Loss=0.1477	acc=0.9537	9410.2 examples/second
[Step=4576]	Loss=0.1478	acc=0.9536	8198.4 examples/second
[Step=4592]	Loss=0.1489	acc=0.9529	8783.9 examples/second
[Step=4608]	Loss=0.1502	acc=0.9526	8722.5 examples/second
[Step=4624]	Loss=0.1508	acc=0.9523	8614.3 examples/second
[Step=4640]	Loss=0.1502	acc=0.9524	10157.3 examples/second
[Step=4656]	Loss=0.1495	acc=0.9526	10681.3 examples/second
[Step=4672]	

[Step=6592]	Loss=0.1274	acc=0.9591	9134.1 examples/second
[Step=6608]	Loss=0.1286	acc=0.9587	8644.7 examples/second
[Step=6624]	Loss=0.1290	acc=0.9584	8684.6 examples/second
[Step=6640]	Loss=0.1287	acc=0.9585	8039.6 examples/second
Test Loss=0.3413, Test acc=0.8989

Epoch: 17
[Step=6656]	Loss=0.1356	acc=0.9549	1029.2 examples/second
[Step=6672]	Loss=0.1215	acc=0.9619	9114.2 examples/second
[Step=6688]	Loss=0.1196	acc=0.9621	9297.9 examples/second
[Step=6704]	Loss=0.1157	acc=0.9650	8803.4 examples/second
[Step=6720]	Loss=0.1132	acc=0.9662	8851.0 examples/second
[Step=6736]	Loss=0.1165	acc=0.9650	8769.5 examples/second
[Step=6752]	Loss=0.1168	acc=0.9647	8964.0 examples/second
[Step=6768]	Loss=0.1177	acc=0.9641	9347.0 examples/second
[Step=6784]	Loss=0.1173	acc=0.9647	8907.7 examples/second
[Step=6800]	Loss=0.1169	acc=0.9645	9961.1 examples/second
[Step=6816]	Loss=0.1183	acc=0.9639	9077.6 examples/second
[Step=6832]	Loss=0.1188	acc=0.9641	8815.8 examples/second
[Step=6848]	Loss=0.1197	acc

[Step=8784]	Loss=0.1120	acc=0.9647	9200.5 examples/second
[Step=8800]	Loss=0.1107	acc=0.9652	8536.3 examples/second
[Step=8816]	Loss=0.1107	acc=0.9651	9056.9 examples/second
[Step=8832]	Loss=0.1094	acc=0.9657	9118.5 examples/second
[Step=8848]	Loss=0.1100	acc=0.9656	8811.2 examples/second
[Step=8864]	Loss=0.1106	acc=0.9651	8738.3 examples/second
[Step=8880]	Loss=0.1106	acc=0.9648	8243.7 examples/second
[Step=8896]	Loss=0.1117	acc=0.9647	9041.4 examples/second
[Step=8912]	Loss=0.1112	acc=0.9647	8940.7 examples/second
[Step=8928]	Loss=0.1108	acc=0.9648	9127.1 examples/second
[Step=8944]	Loss=0.1109	acc=0.9647	8522.7 examples/second
[Step=8960]	Loss=0.1111	acc=0.9647	10187.3 examples/second
[Step=8976]	Loss=0.1112	acc=0.9646	8218.5 examples/second
[Step=8992]	Loss=0.1120	acc=0.9643	7985.7 examples/second
Test Loss=0.3258, Test acc=0.9093
Saving...

Epoch: 23
[Step=9008]	Loss=0.0990	acc=0.9719	1009.8 examples/second
[Step=9024]	Loss=0.0900	acc=0.9710	10126.4 examples/second
[Step=9040]	Los

...

[Step=30400]	Loss=0.0376	acc=0.9878	8663.3 examples/second
[Step=30416]	Loss=0.0374	acc=0.9876	9332.5 examples/second
[Step=30432]	Loss=0.0377	acc=0.9875	8957.8 examples/second
[Step=30448]	Loss=0.0374	acc=0.9876	10585.2 examples/second
[Step=30464]	Loss=0.0373	acc=0.9876	8649.4 examples/second
[Step=30480]	Loss=0.0375	acc=0.9876	9026.7 examples/second
[Step=30496]	Loss=0.0377	acc=0.9875	8092.4 examples/second
Test Loss=0.3921, Test acc=0.9072

Epoch: 78
[Step=30512]	Loss=0.0322	acc=0.9900	1030.8 examples/second
[Step=30528]	Loss=0.0318	acc=0.9901	9246.1 examples/second
[Step=30544]	Loss=0.0353	acc=0.9890	9120.0 examples/second
[Step=30560]	Loss=0.0362	acc=0.9884	8901.1 examples/second
[Step=30576]	Loss=0.0344	acc=0.9891	9017.1 examples/second
[Step=30592]	Loss=0.0338	acc=0.9892	9117.3 examples/second
[Step=30608]	Loss=0.0339	acc=0.9892	8712.0 examples/second
[Step=30624]	Loss=0.0325	acc=0.9898	8450.2 examples/second
[Step=30640]	Loss=0.0315	acc=0.9900	9057.5 examples/second
[Step=3065

[Step=32560]	Loss=0.0328	acc=0.9892	9108.8 examples/second
[Step=32576]	Loss=0.0326	acc=0.9891	8885.8 examples/second
[Step=32592]	Loss=0.0334	acc=0.9888	9201.8 examples/second
[Step=32608]	Loss=0.0338	acc=0.9885	10763.2 examples/second
[Step=32624]	Loss=0.0342	acc=0.9882	8142.0 examples/second
[Step=32640]	Loss=0.0336	acc=0.9883	8950.9 examples/second
[Step=32656]	Loss=0.0334	acc=0.9885	9022.0 examples/second
[Step=32672]	Loss=0.0341	acc=0.9884	8835.6 examples/second
[Step=32688]	Loss=0.0338	acc=0.9885	9197.2 examples/second
[Step=32704]	Loss=0.0339	acc=0.9886	8890.9 examples/second
[Step=32720]	Loss=0.0336	acc=0.9887	9410.1 examples/second
[Step=32736]	Loss=0.0339	acc=0.9887	8638.2 examples/second
[Step=32752]	Loss=0.0340	acc=0.9888	8629.6 examples/second
[Step=32768]	Loss=0.0334	acc=0.9889	9627.4 examples/second
[Step=32784]	Loss=0.0336	acc=0.9889	9762.5 examples/second
[Step=32800]	Loss=0.0339	acc=0.9889	8348.0 examples/second
[Step=32816]	Loss=0.0336	acc=0.9889	8928.8 examples/sec

[Step=34720]	Loss=0.0283	acc=0.9909	9032.7 examples/second
[Step=34736]	Loss=0.0284	acc=0.9909	8449.7 examples/second
[Step=34752]	Loss=0.0287	acc=0.9909	8891.7 examples/second
[Step=34768]	Loss=0.0285	acc=0.9910	9150.5 examples/second
[Step=34784]	Loss=0.0285	acc=0.9910	8835.5 examples/second
Test Loss=0.3949, Test acc=0.9097

Epoch: 89
[Step=34800]	Loss=0.0169	acc=0.9922	1030.9 examples/second
[Step=34816]	Loss=0.0354	acc=0.9890	7935.9 examples/second
[Step=34832]	Loss=0.0328	acc=0.9893	8355.5 examples/second
[Step=34848]	Loss=0.0285	acc=0.9904	8920.0 examples/second
[Step=34864]	Loss=0.0290	acc=0.9906	9019.4 examples/second
[Step=34880]	Loss=0.0286	acc=0.9906	9118.8 examples/second
[Step=34896]	Loss=0.0292	acc=0.9903	8758.5 examples/second
[Step=34912]	Loss=0.0300	acc=0.9898	9270.7 examples/second
[Step=34928]	Loss=0.0298	acc=0.9898	8895.9 examples/second
[Step=34944]	Loss=0.0296	acc=0.9900	7969.1 examples/second
[Step=34960]	Loss=0.0300	acc=0.9900	8249.5 examples/second
[Step=34976

[Step=36880]	Loss=0.0240	acc=0.9928	8582.1 examples/second
[Step=36896]	Loss=0.0250	acc=0.9923	9973.1 examples/second
[Step=36912]	Loss=0.0265	acc=0.9918	9395.9 examples/second
[Step=36928]	Loss=0.0262	acc=0.9917	9045.6 examples/second
[Step=36944]	Loss=0.0268	acc=0.9913	8475.4 examples/second
[Step=36960]	Loss=0.0271	acc=0.9912	9205.8 examples/second
[Step=36976]	Loss=0.0275	acc=0.9911	8611.3 examples/second
[Step=36992]	Loss=0.0279	acc=0.9908	9089.4 examples/second
[Step=37008]	Loss=0.0280	acc=0.9906	10094.6 examples/second
[Step=37024]	Loss=0.0276	acc=0.9907	10228.1 examples/second
[Step=37040]	Loss=0.0281	acc=0.9905	9251.2 examples/second
[Step=37056]	Loss=0.0287	acc=0.9902	9482.5 examples/second
[Step=37072]	Loss=0.0287	acc=0.9902	9386.6 examples/second
[Step=37088]	Loss=0.0287	acc=0.9902	9900.5 examples/second
[Step=37104]	Loss=0.0282	acc=0.9904	9566.0 examples/second
[Step=37120]	Loss=0.0282	acc=0.9905	10034.9 examples/second
[Step=37136]	Loss=0.0279	acc=0.9906	8245.8 examples/s

[Step=39040]	Loss=0.0277	acc=0.9911	8962.9 examples/second
[Step=39056]	Loss=0.0278	acc=0.9912	8680.2 examples/second
[Step=39072]	Loss=0.0279	acc=0.9910	8284.6 examples/second
[Step=39088]	Loss=0.0280	acc=0.9910	8630.4 examples/second
Test Loss=0.3947, Test acc=0.9112


In [7]:
# Load the best weight paramters
net.load_state_dict(torch.load("net_after_pruning.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3773, Test accuracy=0.9141


In [8]:
print("-----Summary After pruning-----")
summary(net)
print("-------------------------------")

-----Summary After pruning-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		475			0.450231
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	9216		5069			0.449978
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	18432		10138			0.449978
8		BatchNorm	N/A		N/A			N/A
9		ReLU		N/A		N/A			N/A
10		Convolutional	36864		20275			0.450005
11		BatchNorm	N/A		N/A			N/A
12		ReLU		N/A		N/A			N/A
13		Convolutional	73728		40550			0.450005
14		BatchNorm	N/A		N/A			N/A
15		ReLU		N/A		N/A			N/A
16		Convolutional	147456		81101			0.449999
17		BatchNorm	N/A		N/A			N/A
18		ReLU		N/A		N/A			N/A
19		Convolutional	147456		81101			0.449999
20		BatchNorm	N/A		N/A			N/A
21		ReLU		N/A		N/A			N/A
22		Convolutional	294912		162202			0.449999
23		BatchNorm	N/A		N/A			N/A
24		ReLU		N/A		N/A			N/A
25		Convolutional	589824		324403			0.450000
26		BatchNorm	N/A		N/A			N/A
27		ReLU		N/A		N/A			N/A
28		Convolutional	589824		324403			0.450000
29		BatchNor

### Quantization

In [9]:
centers = quantize_whole_model(net, bits=3)
np.save("codebook_vgg16.npy", centers)

Complete 1 layers quantization...
Complete 2 layers quantization...
Complete 3 layers quantization...
Complete 4 layers quantization...
Complete 5 layers quantization...
Complete 6 layers quantization...
Complete 7 layers quantization...
Complete 8 layers quantization...
Complete 9 layers quantization...
Complete 10 layers quantization...
Complete 11 layers quantization...
Complete 12 layers quantization...
Complete 13 layers quantization...
Complete 14 layers quantization...
Complete 15 layers quantization...
Complete 16 layers quantization...


In [10]:
#test(net)

In [11]:
# Uncomment to load pretrained weights
#net.load_state_dict(torch.load("net_after_quantization.pt"))
# Comment if you have loaded pretrained weights
finetune_after_quantize(net, epochs=100, batch_size=128, lr=0.001, reg=5e-5)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=16]	Loss=0.0677	acc=0.9800	2155.6 examples/second
[Step=32]	Loss=0.0668	acc=0.9805	9327.8 examples/second
[Step=48]	Loss=0.0650	acc=0.9798	9277.5 examples/second
[Step=64]	Loss=0.0616	acc=0.9811	8441.3 examples/second
[Step=80]	Loss=0.0612	acc=0.9813	9026.7 examples/second
[Step=96]	Loss=0.0598	acc=0.9815	8816.5 examples/second
[Step=112]	Loss=0.0623	acc=0.9812	8524.0 examples/second
[Step=128]	Loss=0.0613	acc=0.9813	8946.8 examples/second
[Step=144]	Loss=0.0597	acc=0.9818	8540.5 examples/second
[Step=160]	Loss=0.0597	acc=0.9816	8980.7 examples/second
[Step=176]	Loss=0.0571	acc=0.9822	8286.7 examples/second
[Step=192]	Loss=0.0563	acc=0.9824	9049.2 examples/second
[Step=208]	Loss=0.0560	acc=0.9824	10369.0 examples/second
[Step=224]	Loss=0.0557	acc=0.9822	8911.5 examples/second
[Step=240]	Loss=0.0559	acc=0.9822	9348.0 examples/second
[Step=256]	Loss=0.0553	acc=0.9822	9196.0 ex

[Step=2208]	Loss=0.0334	acc=0.9892	9266.9 examples/second
[Step=2224]	Loss=0.0326	acc=0.9895	9625.2 examples/second
[Step=2240]	Loss=0.0330	acc=0.9893	9024.9 examples/second
[Step=2256]	Loss=0.0335	acc=0.9891	9318.6 examples/second
[Step=2272]	Loss=0.0341	acc=0.9888	8928.8 examples/second
[Step=2288]	Loss=0.0340	acc=0.9888	8464.8 examples/second
[Step=2304]	Loss=0.0340	acc=0.9887	10388.3 examples/second
[Step=2320]	Loss=0.0338	acc=0.9888	8996.0 examples/second
[Step=2336]	Loss=0.0337	acc=0.9888	8072.6 examples/second
Test Loss=0.3793, Test acc=0.9103
Saving...

Epoch: 6
[Step=2352]	Loss=0.0248	acc=0.9909	1019.6 examples/second
[Step=2368]	Loss=0.0255	acc=0.9911	8445.6 examples/second
[Step=2384]	Loss=0.0252	acc=0.9914	9258.0 examples/second
[Step=2400]	Loss=0.0262	acc=0.9918	9629.6 examples/second
[Step=2416]	Loss=0.0252	acc=0.9920	9141.5 examples/second
[Step=2432]	Loss=0.0244	acc=0.9923	8130.3 examples/second
[Step=2448]	Loss=0.0257	acc=0.9917	9043.2 examples/second
[Step=2464]	Loss=

[Step=4384]	Loss=0.0300	acc=0.9912	9351.4 examples/second
[Step=4400]	Loss=0.0292	acc=0.9912	8801.9 examples/second
[Step=4416]	Loss=0.0285	acc=0.9913	8933.7 examples/second
[Step=4432]	Loss=0.0288	acc=0.9911	8635.0 examples/second
[Step=4448]	Loss=0.0293	acc=0.9908	9129.6 examples/second
[Step=4464]	Loss=0.0296	acc=0.9907	9049.1 examples/second
[Step=4480]	Loss=0.0307	acc=0.9904	8906.5 examples/second
[Step=4496]	Loss=0.0305	acc=0.9903	9216.7 examples/second
[Step=4512]	Loss=0.0313	acc=0.9901	8394.0 examples/second
[Step=4528]	Loss=0.0318	acc=0.9901	9220.8 examples/second
[Step=4544]	Loss=0.0311	acc=0.9903	9205.3 examples/second
[Step=4560]	Loss=0.0314	acc=0.9902	9251.3 examples/second
[Step=4576]	Loss=0.0312	acc=0.9903	9146.5 examples/second
[Step=4592]	Loss=0.0314	acc=0.9903	8993.9 examples/second
[Step=4608]	Loss=0.0312	acc=0.9905	9372.2 examples/second
[Step=4624]	Loss=0.0312	acc=0.9904	10628.2 examples/second
[Step=4640]	Loss=0.0311	acc=0.9906	10641.6 examples/second
[Step=4656]	

[Step=6592]	Loss=0.0267	acc=0.9915	9302.5 examples/second
[Step=6608]	Loss=0.0268	acc=0.9914	8567.9 examples/second
[Step=6624]	Loss=0.0272	acc=0.9914	9072.4 examples/second
[Step=6640]	Loss=0.0276	acc=0.9913	8188.1 examples/second
Test Loss=0.4070, Test acc=0.9065

Epoch: 17
[Step=6656]	Loss=0.0311	acc=0.9896	998.1 examples/second
[Step=6672]	Loss=0.0283	acc=0.9903	8811.3 examples/second
[Step=6688]	Loss=0.0253	acc=0.9916	9287.0 examples/second
[Step=6704]	Loss=0.0264	acc=0.9912	8728.4 examples/second
[Step=6720]	Loss=0.0268	acc=0.9918	9361.5 examples/second
[Step=6736]	Loss=0.0255	acc=0.9925	8502.2 examples/second
[Step=6752]	Loss=0.0262	acc=0.9920	8697.6 examples/second
[Step=6768]	Loss=0.0263	acc=0.9918	8784.3 examples/second
[Step=6784]	Loss=0.0268	acc=0.9917	9223.8 examples/second
[Step=6800]	Loss=0.0262	acc=0.9920	8944.6 examples/second
[Step=6816]	Loss=0.0267	acc=0.9917	9335.4 examples/second
[Step=6832]	Loss=0.0269	acc=0.9917	9047.6 examples/second
[Step=6848]	Loss=0.0269	acc=

[Step=8784]	Loss=0.0257	acc=0.9919	9069.8 examples/second
[Step=8800]	Loss=0.0257	acc=0.9920	9106.0 examples/second
[Step=8816]	Loss=0.0257	acc=0.9920	9646.6 examples/second
[Step=8832]	Loss=0.0252	acc=0.9922	8993.9 examples/second
[Step=8848]	Loss=0.0250	acc=0.9922	8840.5 examples/second
[Step=8864]	Loss=0.0247	acc=0.9922	9004.3 examples/second
[Step=8880]	Loss=0.0243	acc=0.9924	8941.5 examples/second
[Step=8896]	Loss=0.0242	acc=0.9923	8997.2 examples/second
[Step=8912]	Loss=0.0242	acc=0.9924	8965.5 examples/second
[Step=8928]	Loss=0.0241	acc=0.9925	8855.5 examples/second
[Step=8944]	Loss=0.0245	acc=0.9924	8749.2 examples/second
[Step=8960]	Loss=0.0247	acc=0.9924	9668.4 examples/second
[Step=8976]	Loss=0.0247	acc=0.9923	8972.7 examples/second
[Step=8992]	Loss=0.0246	acc=0.9922	7975.5 examples/second
Test Loss=0.3884, Test acc=0.9118

Epoch: 23
[Step=9008]	Loss=0.0182	acc=0.9943	1009.5 examples/second
[Step=9024]	Loss=0.0186	acc=0.9940	9399.6 examples/second
[Step=9040]	Loss=0.0203	acc

...

[Step=30400]	Loss=0.0159	acc=0.9948	8888.2 examples/second
[Step=30416]	Loss=0.0158	acc=0.9949	8756.1 examples/second
[Step=30432]	Loss=0.0155	acc=0.9950	8974.0 examples/second
[Step=30448]	Loss=0.0155	acc=0.9950	9704.8 examples/second
[Step=30464]	Loss=0.0157	acc=0.9950	9213.4 examples/second
[Step=30480]	Loss=0.0157	acc=0.9950	8945.1 examples/second
[Step=30496]	Loss=0.0157	acc=0.9950	8454.2 examples/second
Test Loss=0.4315, Test acc=0.9129

Epoch: 78
[Step=30512]	Loss=0.0112	acc=0.9961	1019.7 examples/second
[Step=30528]	Loss=0.0177	acc=0.9935	9353.3 examples/second
[Step=30544]	Loss=0.0180	acc=0.9935	8514.5 examples/second
[Step=30560]	Loss=0.0179	acc=0.9931	9014.9 examples/second
[Step=30576]	Loss=0.0167	acc=0.9935	10263.6 examples/second
[Step=30592]	Loss=0.0162	acc=0.9939	8971.6 examples/second
[Step=30608]	Loss=0.0164	acc=0.9942	8155.1 examples/second
[Step=30624]	Loss=0.0156	acc=0.9944	8662.9 examples/second
[Step=30640]	Loss=0.0153	acc=0.9946	9010.8 examples/second
[Step=3065

[Step=32560]	Loss=0.0110	acc=0.9965	10064.4 examples/second
[Step=32576]	Loss=0.0113	acc=0.9963	8946.6 examples/second
[Step=32592]	Loss=0.0118	acc=0.9963	9236.4 examples/second
[Step=32608]	Loss=0.0115	acc=0.9964	8732.4 examples/second
[Step=32624]	Loss=0.0112	acc=0.9964	9395.1 examples/second
[Step=32640]	Loss=0.0114	acc=0.9964	8974.6 examples/second
[Step=32656]	Loss=0.0115	acc=0.9962	9697.6 examples/second
[Step=32672]	Loss=0.0115	acc=0.9963	9290.2 examples/second
[Step=32688]	Loss=0.0117	acc=0.9962	9387.5 examples/second
[Step=32704]	Loss=0.0114	acc=0.9963	8006.3 examples/second
[Step=32720]	Loss=0.0120	acc=0.9961	8838.0 examples/second
[Step=32736]	Loss=0.0121	acc=0.9962	8717.5 examples/second
[Step=32752]	Loss=0.0123	acc=0.9961	8805.9 examples/second
[Step=32768]	Loss=0.0127	acc=0.9960	9883.2 examples/second
[Step=32784]	Loss=0.0125	acc=0.9961	9769.3 examples/second
[Step=32800]	Loss=0.0126	acc=0.9962	8523.8 examples/second
[Step=32816]	Loss=0.0128	acc=0.9961	9222.7 examples/sec

[Step=34720]	Loss=0.0147	acc=0.9954	8897.7 examples/second
[Step=34736]	Loss=0.0146	acc=0.9954	8711.0 examples/second
[Step=34752]	Loss=0.0143	acc=0.9955	9355.8 examples/second
[Step=34768]	Loss=0.0144	acc=0.9954	9102.0 examples/second
[Step=34784]	Loss=0.0146	acc=0.9954	8315.0 examples/second
Test Loss=0.4196, Test acc=0.9141

Epoch: 89
[Step=34800]	Loss=0.0047	acc=1.0000	1015.7 examples/second
[Step=34816]	Loss=0.0099	acc=0.9959	9304.8 examples/second
[Step=34832]	Loss=0.0146	acc=0.9950	8611.5 examples/second
[Step=34848]	Loss=0.0158	acc=0.9949	8629.3 examples/second
[Step=34864]	Loss=0.0172	acc=0.9948	9044.8 examples/second
[Step=34880]	Loss=0.0154	acc=0.9954	9405.5 examples/second
[Step=34896]	Loss=0.0149	acc=0.9957	8999.2 examples/second
[Step=34912]	Loss=0.0148	acc=0.9956	8862.6 examples/second
[Step=34928]	Loss=0.0138	acc=0.9959	8084.1 examples/second
[Step=34944]	Loss=0.0139	acc=0.9958	8880.4 examples/second
[Step=34960]	Loss=0.0135	acc=0.9960	8871.4 examples/second
[Step=34976

[Step=36880]	Loss=0.0110	acc=0.9963	8953.4 examples/second
[Step=36896]	Loss=0.0125	acc=0.9958	9056.7 examples/second
[Step=36912]	Loss=0.0135	acc=0.9957	8868.1 examples/second
[Step=36928]	Loss=0.0136	acc=0.9959	9100.2 examples/second
[Step=36944]	Loss=0.0137	acc=0.9958	8827.1 examples/second
[Step=36960]	Loss=0.0138	acc=0.9958	9342.7 examples/second
[Step=36976]	Loss=0.0142	acc=0.9957	9058.6 examples/second
[Step=36992]	Loss=0.0139	acc=0.9957	10774.2 examples/second
[Step=37008]	Loss=0.0138	acc=0.9957	8825.3 examples/second
[Step=37024]	Loss=0.0137	acc=0.9958	9057.7 examples/second
[Step=37040]	Loss=0.0137	acc=0.9958	8535.0 examples/second
[Step=37056]	Loss=0.0140	acc=0.9957	9035.7 examples/second
[Step=37072]	Loss=0.0141	acc=0.9958	9252.0 examples/second
[Step=37088]	Loss=0.0143	acc=0.9957	9011.8 examples/second
[Step=37104]	Loss=0.0142	acc=0.9957	8884.1 examples/second
[Step=37120]	Loss=0.0139	acc=0.9958	8994.7 examples/second
[Step=37136]	Loss=0.0137	acc=0.9958	8344.1 examples/sec

[Step=39040]	Loss=0.0123	acc=0.9960	8704.4 examples/second
[Step=39056]	Loss=0.0121	acc=0.9961	8793.8 examples/second
[Step=39072]	Loss=0.0119	acc=0.9961	8799.4 examples/second
[Step=39088]	Loss=0.0120	acc=0.9961	8562.8 examples/second
Test Loss=0.4230, Test acc=0.9149


In [12]:
# Load the best weight paramters
net.load_state_dict(torch.load("net_after_quantization.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.4022, Test accuracy=0.9190


In [13]:
print("-----Summary After pruning-----")
summary(net)
print("-------------------------------")

-----Summary After pruning-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		475			0.450231
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	9216		5069			0.449978
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	18432		10138			0.449978
8		BatchNorm	N/A		N/A			N/A
9		ReLU		N/A		N/A			N/A
10		Convolutional	36864		20275			0.450005
11		BatchNorm	N/A		N/A			N/A
12		ReLU		N/A		N/A			N/A
13		Convolutional	73728		40550			0.450005
14		BatchNorm	N/A		N/A			N/A
15		ReLU		N/A		N/A			N/A
16		Convolutional	147456		81101			0.449999
17		BatchNorm	N/A		N/A			N/A
18		ReLU		N/A		N/A			N/A
19		Convolutional	147456		81101			0.449999
20		BatchNorm	N/A		N/A			N/A
21		ReLU		N/A		N/A			N/A
22		Convolutional	294912		162202			0.449999
23		BatchNorm	N/A		N/A			N/A
24		ReLU		N/A		N/A			N/A
25		Convolutional	589824		324403			0.450000
26		BatchNorm	N/A		N/A			N/A
27		ReLU		N/A		N/A			N/A
28		Convolutional	589824		324403			0.450000
29		BatchNor

In [14]:
# Test accuracy before fine-tuning
prune(net, method='percentage', q=55.0, s=0.75)
test(net)

Files already downloaded and verified
Test Loss=0.4038, Test accuracy=0.9169


In [15]:
# Uncomment to load pretrained weights
#net.load_state_dict(torch.load("net_after_pruning.pt"))
# Comment if you have loaded pretrained weights
finetune_after_prune(net, epochs=100, batch_size=128, lr=0.001, reg=5e-5)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=16]	Loss=0.0092	acc=0.9971	2351.4 examples/second
[Step=32]	Loss=0.0121	acc=0.9961	9533.8 examples/second
[Step=48]	Loss=0.0125	acc=0.9953	9140.3 examples/second
[Step=64]	Loss=0.0142	acc=0.9949	8622.0 examples/second
[Step=80]	Loss=0.0145	acc=0.9949	8796.7 examples/second
[Step=96]	Loss=0.0140	acc=0.9952	8869.0 examples/second
[Step=112]	Loss=0.0133	acc=0.9955	8426.9 examples/second
[Step=128]	Loss=0.0131	acc=0.9954	9374.1 examples/second
[Step=144]	Loss=0.0130	acc=0.9955	9040.8 examples/second
[Step=160]	Loss=0.0129	acc=0.9955	9255.5 examples/second
[Step=176]	Loss=0.0130	acc=0.9955	9188.4 examples/second
[Step=192]	Loss=0.0129	acc=0.9955	8985.6 examples/second
[Step=208]	Loss=0.0131	acc=0.9955	9593.4 examples/second
[Step=224]	Loss=0.0134	acc=0.9954	8837.6 examples/second
[Step=240]	Loss=0.0134	acc=0.9954	7972.0 examples/second
[Step=256]	Loss=0.0138	acc=0.9953	8538.6 exa

[Step=2208]	Loss=0.0113	acc=0.9964	8857.7 examples/second
[Step=2224]	Loss=0.0116	acc=0.9963	8720.2 examples/second
[Step=2240]	Loss=0.0115	acc=0.9962	9400.8 examples/second
[Step=2256]	Loss=0.0118	acc=0.9961	9579.2 examples/second
[Step=2272]	Loss=0.0117	acc=0.9961	8651.1 examples/second
[Step=2288]	Loss=0.0119	acc=0.9960	8317.7 examples/second
[Step=2304]	Loss=0.0119	acc=0.9961	7943.1 examples/second
[Step=2320]	Loss=0.0118	acc=0.9961	10533.3 examples/second
[Step=2336]	Loss=0.0117	acc=0.9961	8152.9 examples/second
Test Loss=0.4163, Test acc=0.9165

Epoch: 6
[Step=2352]	Loss=0.0044	acc=0.9987	1029.1 examples/second
[Step=2368]	Loss=0.0092	acc=0.9975	9409.7 examples/second
[Step=2384]	Loss=0.0089	acc=0.9975	9912.2 examples/second
[Step=2400]	Loss=0.0080	acc=0.9978	9801.6 examples/second
[Step=2416]	Loss=0.0081	acc=0.9974	8657.5 examples/second
[Step=2432]	Loss=0.0093	acc=0.9971	8860.6 examples/second
[Step=2448]	Loss=0.0100	acc=0.9969	9520.7 examples/second
[Step=2464]	Loss=0.0098	acc

[Step=4400]	Loss=0.0106	acc=0.9964	8435.0 examples/second
[Step=4416]	Loss=0.0110	acc=0.9963	8933.4 examples/second
[Step=4432]	Loss=0.0110	acc=0.9963	8343.9 examples/second
[Step=4448]	Loss=0.0111	acc=0.9962	9189.8 examples/second
[Step=4464]	Loss=0.0114	acc=0.9961	8706.8 examples/second
[Step=4480]	Loss=0.0113	acc=0.9962	8103.9 examples/second
[Step=4496]	Loss=0.0110	acc=0.9963	9399.1 examples/second
[Step=4512]	Loss=0.0109	acc=0.9964	9513.8 examples/second
[Step=4528]	Loss=0.0109	acc=0.9965	9456.1 examples/second
[Step=4544]	Loss=0.0112	acc=0.9963	9519.9 examples/second
[Step=4560]	Loss=0.0113	acc=0.9963	8299.3 examples/second
[Step=4576]	Loss=0.0114	acc=0.9963	8508.6 examples/second
[Step=4592]	Loss=0.0112	acc=0.9964	8552.2 examples/second
[Step=4608]	Loss=0.0111	acc=0.9964	8676.2 examples/second
[Step=4624]	Loss=0.0111	acc=0.9964	8599.9 examples/second
[Step=4640]	Loss=0.0111	acc=0.9963	7820.1 examples/second
[Step=4656]	Loss=0.0109	acc=0.9963	9204.4 examples/second
[Step=4672]	Lo

[Step=6608]	Loss=0.0102	acc=0.9968	9680.7 examples/second
[Step=6624]	Loss=0.0105	acc=0.9967	9865.3 examples/second
[Step=6640]	Loss=0.0106	acc=0.9967	8322.9 examples/second
Test Loss=0.4181, Test acc=0.9166

Epoch: 17
[Step=6656]	Loss=0.0152	acc=0.9957	1035.8 examples/second
[Step=6672]	Loss=0.0101	acc=0.9962	8535.6 examples/second
[Step=6688]	Loss=0.0085	acc=0.9971	9930.8 examples/second
[Step=6704]	Loss=0.0078	acc=0.9973	8579.0 examples/second
[Step=6720]	Loss=0.0083	acc=0.9970	9047.0 examples/second
[Step=6736]	Loss=0.0080	acc=0.9972	9919.1 examples/second
[Step=6752]	Loss=0.0088	acc=0.9969	9073.5 examples/second
[Step=6768]	Loss=0.0088	acc=0.9968	8950.1 examples/second
[Step=6784]	Loss=0.0089	acc=0.9967	8663.3 examples/second
[Step=6800]	Loss=0.0091	acc=0.9968	9077.1 examples/second
[Step=6816]	Loss=0.0095	acc=0.9967	10349.7 examples/second
[Step=6832]	Loss=0.0097	acc=0.9966	9255.0 examples/second
[Step=6848]	Loss=0.0103	acc=0.9964	8808.5 examples/second
[Step=6864]	Loss=0.0102	ac

[Step=8800]	Loss=0.0093	acc=0.9974	8640.9 examples/second
[Step=8816]	Loss=0.0093	acc=0.9973	8791.1 examples/second
[Step=8832]	Loss=0.0090	acc=0.9974	8723.3 examples/second
[Step=8848]	Loss=0.0089	acc=0.9974	8446.5 examples/second
[Step=8864]	Loss=0.0088	acc=0.9974	7412.3 examples/second
[Step=8880]	Loss=0.0089	acc=0.9974	8719.9 examples/second
[Step=8896]	Loss=0.0091	acc=0.9972	8634.5 examples/second
[Step=8912]	Loss=0.0092	acc=0.9972	8054.4 examples/second
[Step=8928]	Loss=0.0094	acc=0.9971	10720.5 examples/second
[Step=8944]	Loss=0.0094	acc=0.9971	10116.9 examples/second
[Step=8960]	Loss=0.0097	acc=0.9969	8922.3 examples/second
[Step=8976]	Loss=0.0099	acc=0.9969	8666.9 examples/second
[Step=8992]	Loss=0.0101	acc=0.9969	7918.6 examples/second
Test Loss=0.4264, Test acc=0.9156

Epoch: 23
[Step=9008]	Loss=0.0109	acc=0.9974	1028.2 examples/second
[Step=9024]	Loss=0.0093	acc=0.9975	9049.3 examples/second
[Step=9040]	Loss=0.0097	acc=0.9970	8928.0 examples/second
[Step=9056]	Loss=0.0099	a

...

[Step=30384]	Loss=0.0076	acc=0.9976	8872.4 examples/second
[Step=30400]	Loss=0.0082	acc=0.9974	8905.6 examples/second
[Step=30416]	Loss=0.0082	acc=0.9974	8887.6 examples/second
[Step=30432]	Loss=0.0080	acc=0.9975	9557.3 examples/second
[Step=30448]	Loss=0.0081	acc=0.9974	8789.6 examples/second
[Step=30464]	Loss=0.0080	acc=0.9975	9142.5 examples/second
[Step=30480]	Loss=0.0079	acc=0.9975	8651.7 examples/second
[Step=30496]	Loss=0.0081	acc=0.9975	8359.7 examples/second
Test Loss=0.4526, Test acc=0.9151

Epoch: 78
[Step=30512]	Loss=0.0021	acc=1.0000	1040.2 examples/second
[Step=30528]	Loss=0.0026	acc=0.9997	10026.6 examples/second
[Step=30544]	Loss=0.0045	acc=0.9990	8748.3 examples/second
[Step=30560]	Loss=0.0065	acc=0.9980	8859.2 examples/second
[Step=30576]	Loss=0.0070	acc=0.9978	8328.9 examples/second
[Step=30592]	Loss=0.0078	acc=0.9978	8361.2 examples/second
[Step=30608]	Loss=0.0078	acc=0.9977	9589.9 examples/second
[Step=30624]	Loss=0.0086	acc=0.9975	10579.1 examples/second
[Step=306

[Step=32544]	Loss=0.0063	acc=0.9978	8671.8 examples/second
[Step=32560]	Loss=0.0068	acc=0.9978	8952.5 examples/second
[Step=32576]	Loss=0.0070	acc=0.9976	8555.8 examples/second
[Step=32592]	Loss=0.0070	acc=0.9978	8803.7 examples/second
[Step=32608]	Loss=0.0068	acc=0.9978	8736.0 examples/second
[Step=32624]	Loss=0.0072	acc=0.9977	8795.9 examples/second
[Step=32640]	Loss=0.0071	acc=0.9977	8810.6 examples/second
[Step=32656]	Loss=0.0071	acc=0.9976	9081.6 examples/second
[Step=32672]	Loss=0.0070	acc=0.9976	8603.1 examples/second
[Step=32688]	Loss=0.0074	acc=0.9974	8883.3 examples/second
[Step=32704]	Loss=0.0073	acc=0.9974	9439.4 examples/second
[Step=32720]	Loss=0.0074	acc=0.9974	8904.9 examples/second
[Step=32736]	Loss=0.0075	acc=0.9974	8947.8 examples/second
[Step=32752]	Loss=0.0077	acc=0.9973	8853.2 examples/second
[Step=32768]	Loss=0.0076	acc=0.9974	9178.4 examples/second
[Step=32784]	Loss=0.0074	acc=0.9975	8589.1 examples/second
[Step=32800]	Loss=0.0073	acc=0.9975	8650.6 examples/seco

[Step=34704]	Loss=0.0067	acc=0.9976	8351.9 examples/second
[Step=34720]	Loss=0.0066	acc=0.9977	8930.5 examples/second
[Step=34736]	Loss=0.0068	acc=0.9977	9462.1 examples/second
[Step=34752]	Loss=0.0069	acc=0.9976	9796.3 examples/second
[Step=34768]	Loss=0.0072	acc=0.9976	10647.9 examples/second
[Step=34784]	Loss=0.0073	acc=0.9976	8735.4 examples/second
Test Loss=0.4493, Test acc=0.9166

Epoch: 89
[Step=34800]	Loss=0.0014	acc=1.0000	1034.9 examples/second
[Step=34816]	Loss=0.0141	acc=0.9968	7160.2 examples/second
[Step=34832]	Loss=0.0115	acc=0.9967	9016.9 examples/second
[Step=34848]	Loss=0.0120	acc=0.9963	9270.3 examples/second
[Step=34864]	Loss=0.0107	acc=0.9970	9639.2 examples/second
[Step=34880]	Loss=0.0101	acc=0.9972	8753.4 examples/second
[Step=34896]	Loss=0.0104	acc=0.9969	9051.6 examples/second
[Step=34912]	Loss=0.0100	acc=0.9970	8957.2 examples/second
[Step=34928]	Loss=0.0106	acc=0.9967	9588.3 examples/second
[Step=34944]	Loss=0.0108	acc=0.9966	10744.5 examples/second
[Step=349

[Step=36864]	Loss=0.0079	acc=0.9973	8815.6 examples/second
[Step=36880]	Loss=0.0076	acc=0.9974	9055.7 examples/second
[Step=36896]	Loss=0.0079	acc=0.9975	8863.7 examples/second
[Step=36912]	Loss=0.0078	acc=0.9975	9112.1 examples/second
[Step=36928]	Loss=0.0079	acc=0.9975	8222.8 examples/second
[Step=36944]	Loss=0.0080	acc=0.9975	8357.0 examples/second
[Step=36960]	Loss=0.0077	acc=0.9976	8520.9 examples/second
[Step=36976]	Loss=0.0078	acc=0.9975	8506.9 examples/second
[Step=36992]	Loss=0.0075	acc=0.9976	8630.3 examples/second
[Step=37008]	Loss=0.0073	acc=0.9976	8596.0 examples/second
[Step=37024]	Loss=0.0074	acc=0.9976	9024.6 examples/second
[Step=37040]	Loss=0.0072	acc=0.9976	10676.7 examples/second
[Step=37056]	Loss=0.0071	acc=0.9977	8669.3 examples/second
[Step=37072]	Loss=0.0071	acc=0.9977	10227.5 examples/second
[Step=37088]	Loss=0.0071	acc=0.9976	8426.9 examples/second
[Step=37104]	Loss=0.0070	acc=0.9976	9373.9 examples/second
[Step=37120]	Loss=0.0071	acc=0.9975	9341.9 examples/se

[Step=39024]	Loss=0.0082	acc=0.9972	8658.5 examples/second
[Step=39040]	Loss=0.0085	acc=0.9972	8428.9 examples/second
[Step=39056]	Loss=0.0086	acc=0.9971	9006.1 examples/second
[Step=39072]	Loss=0.0084	acc=0.9971	9495.6 examples/second
[Step=39088]	Loss=0.0084	acc=0.9971	8324.4 examples/second
Test Loss=0.4423, Test acc=0.9178


In [16]:
# Load the best weight paramters
net.load_state_dict(torch.load("net_after_pruning.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.4347, Test accuracy=0.9194


### Huffman Coding

In [17]:
frequency_map, encoding_map = huffman_coding(net, centers)
np.save("huffman_encoding", encoding_map)
np.save("huffman_freq", frequency_map)

Original storage for each parameter: 3.0000 bits
Average storage for each parameter after Huffman Coding: 2.5000 bits
Complete 1 layers for Huffman Coding...
Original storage for each parameter: 3.0000 bits
Average storage for each parameter after Huffman Coding: 2.5000 bits
Complete 2 layers for Huffman Coding...
Original storage for each parameter: 3.0000 bits
Average storage for each parameter after Huffman Coding: 2.5000 bits
Complete 3 layers for Huffman Coding...
Original storage for each parameter: 3.0000 bits
Average storage for each parameter after Huffman Coding: 2.5000 bits
Complete 4 layers for Huffman Coding...
Original storage for each parameter: 3.0000 bits
Average storage for each parameter after Huffman Coding: 2.5000 bits
Complete 5 layers for Huffman Coding...
Original storage for each parameter: 3.0000 bits
Average storage for each parameter after Huffman Coding: 2.5000 bits
Complete 6 layers for Huffman Coding...
Original storage for each parameter: 3.0000 bits
Ave

In [18]:
test(net)

Files already downloaded and verified
Test Loss=0.4347, Test accuracy=0.9194


In [19]:
# Test accuracy before fine-tuning
prune(net, method='percentage', q=75.0, s=0.75)
test(net)

Files already downloaded and verified
Test Loss=0.7431, Test accuracy=0.8232


In [20]:
# Uncomment to load pretrained weights
#net.load_state_dict(torch.load("net_after_pruning.pt"))
# Comment if you have loaded pretrained weights
finetune_after_prune(net, epochs=100, batch_size=128, lr=0.001, reg=5e-5)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=16]	Loss=0.1467	acc=0.9595	2254.2 examples/second
[Step=32]	Loss=0.1394	acc=0.9624	9168.0 examples/second
[Step=48]	Loss=0.1275	acc=0.9645	9780.9 examples/second
[Step=64]	Loss=0.1155	acc=0.9670	9067.3 examples/second
[Step=80]	Loss=0.1101	acc=0.9677	9923.7 examples/second
[Step=96]	Loss=0.1039	acc=0.9693	10335.5 examples/second
[Step=112]	Loss=0.0995	acc=0.9699	9793.4 examples/second
[Step=128]	Loss=0.0956	acc=0.9711	8707.4 examples/second
[Step=144]	Loss=0.0936	acc=0.9721	9251.8 examples/second
[Step=160]	Loss=0.0930	acc=0.9720	9345.3 examples/second
[Step=176]	Loss=0.0928	acc=0.9721	9147.0 examples/second
[Step=192]	Loss=0.0911	acc=0.9724	9859.1 examples/second
[Step=208]	Loss=0.0900	acc=0.9725	8554.4 examples/second
[Step=224]	Loss=0.0882	acc=0.9729	9735.7 examples/second
[Step=240]	Loss=0.0862	acc=0.9735	10757.2 examples/second
[Step=256]	Loss=0.0842	acc=0.9739	7768.5 e

[Step=2208]	Loss=0.0258	acc=0.9915	9331.1 examples/second
[Step=2224]	Loss=0.0258	acc=0.9914	9159.1 examples/second
[Step=2240]	Loss=0.0259	acc=0.9913	9098.0 examples/second
[Step=2256]	Loss=0.0256	acc=0.9913	9236.3 examples/second
[Step=2272]	Loss=0.0257	acc=0.9913	8981.9 examples/second
[Step=2288]	Loss=0.0258	acc=0.9914	8843.6 examples/second
[Step=2304]	Loss=0.0255	acc=0.9915	9200.8 examples/second
[Step=2320]	Loss=0.0252	acc=0.9916	8591.5 examples/second
[Step=2336]	Loss=0.0252	acc=0.9916	8469.6 examples/second
Test Loss=0.4287, Test acc=0.9102
Saving...

Epoch: 6
[Step=2352]	Loss=0.0243	acc=0.9909	983.2 examples/second
[Step=2368]	Loss=0.0299	acc=0.9922	8886.1 examples/second
[Step=2384]	Loss=0.0268	acc=0.9926	9061.4 examples/second
[Step=2400]	Loss=0.0236	acc=0.9931	8885.6 examples/second
[Step=2416]	Loss=0.0245	acc=0.9924	8870.4 examples/second
[Step=2432]	Loss=0.0228	acc=0.9930	9100.9 examples/second
[Step=2448]	Loss=0.0226	acc=0.9930	9005.1 examples/second
[Step=2464]	Loss=0.

[Step=4400]	Loss=0.0172	acc=0.9939	9474.7 examples/second
[Step=4416]	Loss=0.0176	acc=0.9938	9039.4 examples/second
[Step=4432]	Loss=0.0182	acc=0.9936	10498.4 examples/second
[Step=4448]	Loss=0.0190	acc=0.9931	8982.9 examples/second
[Step=4464]	Loss=0.0191	acc=0.9932	9207.8 examples/second
[Step=4480]	Loss=0.0183	acc=0.9935	9401.2 examples/second
[Step=4496]	Loss=0.0182	acc=0.9935	8371.4 examples/second
[Step=4512]	Loss=0.0182	acc=0.9935	9073.4 examples/second
[Step=4528]	Loss=0.0181	acc=0.9935	9892.3 examples/second
[Step=4544]	Loss=0.0183	acc=0.9934	8215.1 examples/second
[Step=4560]	Loss=0.0182	acc=0.9935	9717.5 examples/second
[Step=4576]	Loss=0.0186	acc=0.9934	7970.6 examples/second
[Step=4592]	Loss=0.0185	acc=0.9934	7658.8 examples/second
[Step=4608]	Loss=0.0183	acc=0.9935	9100.9 examples/second
[Step=4624]	Loss=0.0188	acc=0.9934	8975.5 examples/second
[Step=4640]	Loss=0.0189	acc=0.9933	9811.5 examples/second
[Step=4656]	Loss=0.0186	acc=0.9934	9447.6 examples/second
[Step=4672]	L

[Step=6592]	Loss=0.0179	acc=0.9943	9282.8 examples/second
[Step=6608]	Loss=0.0177	acc=0.9944	8128.8 examples/second
[Step=6624]	Loss=0.0174	acc=0.9945	8779.9 examples/second
[Step=6640]	Loss=0.0174	acc=0.9945	8249.8 examples/second
Test Loss=0.4415, Test acc=0.9094

Epoch: 17
[Step=6656]	Loss=0.0166	acc=0.9948	1019.9 examples/second
[Step=6672]	Loss=0.0179	acc=0.9938	8831.9 examples/second
[Step=6688]	Loss=0.0164	acc=0.9945	8376.9 examples/second
[Step=6704]	Loss=0.0161	acc=0.9949	9039.0 examples/second
[Step=6720]	Loss=0.0165	acc=0.9948	9170.0 examples/second
[Step=6736]	Loss=0.0167	acc=0.9946	9096.4 examples/second
[Step=6752]	Loss=0.0167	acc=0.9946	9328.1 examples/second
[Step=6768]	Loss=0.0166	acc=0.9945	9115.0 examples/second
[Step=6784]	Loss=0.0180	acc=0.9941	9846.9 examples/second
[Step=6800]	Loss=0.0180	acc=0.9941	8085.4 examples/second
[Step=6816]	Loss=0.0176	acc=0.9942	9073.9 examples/second
[Step=6832]	Loss=0.0183	acc=0.9939	8457.1 examples/second
[Step=6848]	Loss=0.0181	acc

[Step=8784]	Loss=0.0159	acc=0.9944	9315.0 examples/second
[Step=8800]	Loss=0.0156	acc=0.9946	9838.9 examples/second
[Step=8816]	Loss=0.0155	acc=0.9946	9245.0 examples/second
[Step=8832]	Loss=0.0153	acc=0.9946	8630.7 examples/second
[Step=8848]	Loss=0.0151	acc=0.9947	8522.7 examples/second
[Step=8864]	Loss=0.0153	acc=0.9945	8797.7 examples/second
[Step=8880]	Loss=0.0152	acc=0.9947	8988.2 examples/second
[Step=8896]	Loss=0.0153	acc=0.9946	8871.4 examples/second
[Step=8912]	Loss=0.0155	acc=0.9946	9567.2 examples/second
[Step=8928]	Loss=0.0155	acc=0.9946	9259.8 examples/second
[Step=8944]	Loss=0.0157	acc=0.9946	9088.0 examples/second
[Step=8960]	Loss=0.0155	acc=0.9946	9197.2 examples/second
[Step=8976]	Loss=0.0157	acc=0.9947	9860.1 examples/second
[Step=8992]	Loss=0.0156	acc=0.9947	8723.9 examples/second
Test Loss=0.4301, Test acc=0.9133
Saving...

Epoch: 23
[Step=9008]	Loss=0.0137	acc=0.9948	1000.7 examples/second
[Step=9024]	Loss=0.0133	acc=0.9952	9023.9 examples/second
[Step=9040]	Loss=

...

[Step=30368]	Loss=0.0097	acc=0.9972	8946.5 examples/second
[Step=30384]	Loss=0.0099	acc=0.9970	8470.5 examples/second
[Step=30400]	Loss=0.0095	acc=0.9971	9756.1 examples/second
[Step=30416]	Loss=0.0095	acc=0.9971	9982.1 examples/second
[Step=30432]	Loss=0.0097	acc=0.9971	9300.4 examples/second
[Step=30448]	Loss=0.0100	acc=0.9971	8664.8 examples/second
[Step=30464]	Loss=0.0099	acc=0.9970	9474.4 examples/second
[Step=30480]	Loss=0.0098	acc=0.9971	8292.8 examples/second
[Step=30496]	Loss=0.0098	acc=0.9970	8011.2 examples/second
Test Loss=0.4367, Test acc=0.9147

Epoch: 78
[Step=30512]	Loss=0.0068	acc=0.9978	1003.4 examples/second
[Step=30528]	Loss=0.0071	acc=0.9971	9848.1 examples/second
[Step=30544]	Loss=0.0085	acc=0.9964	8919.7 examples/second
[Step=30560]	Loss=0.0080	acc=0.9968	8535.8 examples/second
[Step=30576]	Loss=0.0094	acc=0.9965	10023.2 examples/second
[Step=30592]	Loss=0.0097	acc=0.9966	10552.3 examples/second
[Step=30608]	Loss=0.0093	acc=0.9967	7989.3 examples/second
[Step=306

[Step=32528]	Loss=0.0061	acc=0.9981	9352.9 examples/second
[Step=32544]	Loss=0.0062	acc=0.9981	9131.8 examples/second
[Step=32560]	Loss=0.0063	acc=0.9980	8746.6 examples/second
[Step=32576]	Loss=0.0067	acc=0.9978	9033.8 examples/second
[Step=32592]	Loss=0.0076	acc=0.9976	10750.7 examples/second
[Step=32608]	Loss=0.0078	acc=0.9975	9090.7 examples/second
[Step=32624]	Loss=0.0080	acc=0.9974	8890.7 examples/second
[Step=32640]	Loss=0.0081	acc=0.9975	9080.2 examples/second
[Step=32656]	Loss=0.0079	acc=0.9975	8954.2 examples/second
[Step=32672]	Loss=0.0080	acc=0.9975	8897.4 examples/second
[Step=32688]	Loss=0.0082	acc=0.9975	8918.2 examples/second
[Step=32704]	Loss=0.0083	acc=0.9975	8897.2 examples/second
[Step=32720]	Loss=0.0083	acc=0.9975	9370.3 examples/second
[Step=32736]	Loss=0.0081	acc=0.9975	9035.9 examples/second
[Step=32752]	Loss=0.0085	acc=0.9974	8859.4 examples/second
[Step=32768]	Loss=0.0084	acc=0.9974	8545.1 examples/second
[Step=32784]	Loss=0.0084	acc=0.9974	8647.2 examples/sec

[Step=34688]	Loss=0.0098	acc=0.9971	9291.6 examples/second
[Step=34704]	Loss=0.0096	acc=0.9971	9316.5 examples/second
[Step=34720]	Loss=0.0095	acc=0.9972	9145.4 examples/second
[Step=34736]	Loss=0.0094	acc=0.9972	9356.8 examples/second
[Step=34752]	Loss=0.0092	acc=0.9972	9246.3 examples/second
[Step=34768]	Loss=0.0094	acc=0.9971	9864.5 examples/second
[Step=34784]	Loss=0.0094	acc=0.9971	8346.0 examples/second
Test Loss=0.4422, Test acc=0.9155

Epoch: 89
[Step=34800]	Loss=0.0100	acc=1.0000	1011.8 examples/second
[Step=34816]	Loss=0.0080	acc=0.9977	7745.9 examples/second
[Step=34832]	Loss=0.0082	acc=0.9976	10347.4 examples/second
[Step=34848]	Loss=0.0093	acc=0.9974	9402.3 examples/second
[Step=34864]	Loss=0.0084	acc=0.9975	8829.0 examples/second
[Step=34880]	Loss=0.0098	acc=0.9972	8895.7 examples/second
[Step=34896]	Loss=0.0091	acc=0.9975	9033.0 examples/second
[Step=34912]	Loss=0.0088	acc=0.9975	8923.6 examples/second
[Step=34928]	Loss=0.0084	acc=0.9976	10610.1 examples/second
[Step=349

[Step=36848]	Loss=0.0108	acc=0.9963	8466.2 examples/second
[Step=36864]	Loss=0.0106	acc=0.9964	9313.0 examples/second
[Step=36880]	Loss=0.0108	acc=0.9963	8608.8 examples/second
[Step=36896]	Loss=0.0110	acc=0.9961	8918.8 examples/second
[Step=36912]	Loss=0.0114	acc=0.9961	9207.3 examples/second
[Step=36928]	Loss=0.0115	acc=0.9960	9041.0 examples/second
[Step=36944]	Loss=0.0113	acc=0.9962	9028.4 examples/second
[Step=36960]	Loss=0.0110	acc=0.9963	9157.6 examples/second
[Step=36976]	Loss=0.0111	acc=0.9963	9280.1 examples/second
[Step=36992]	Loss=0.0106	acc=0.9965	9292.1 examples/second
[Step=37008]	Loss=0.0107	acc=0.9964	8790.1 examples/second
[Step=37024]	Loss=0.0106	acc=0.9964	10578.3 examples/second
[Step=37040]	Loss=0.0110	acc=0.9963	8683.1 examples/second
[Step=37056]	Loss=0.0110	acc=0.9962	8645.0 examples/second
[Step=37072]	Loss=0.0111	acc=0.9963	9847.7 examples/second
[Step=37088]	Loss=0.0110	acc=0.9963	9389.1 examples/second
[Step=37104]	Loss=0.0112	acc=0.9963	7974.5 examples/sec

[Step=39008]	Loss=0.0087	acc=0.9969	10719.0 examples/second
[Step=39024]	Loss=0.0089	acc=0.9968	8975.8 examples/second
[Step=39040]	Loss=0.0090	acc=0.9967	9831.0 examples/second
[Step=39056]	Loss=0.0089	acc=0.9968	9627.2 examples/second
[Step=39072]	Loss=0.0090	acc=0.9967	9529.7 examples/second
[Step=39088]	Loss=0.0089	acc=0.9967	8333.8 examples/second
Test Loss=0.4538, Test acc=0.9161


In [21]:
# Load the best weight paramters
net.load_state_dict(torch.load("net_after_pruning.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.4334, Test accuracy=0.9185


In [22]:
print("-----Summary After pruning-----")
summary(net)
print("-------------------------------")

-----Summary After pruning-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		216			0.750000
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	9216		2304			0.750000
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	18432		4608			0.750000
8		BatchNorm	N/A		N/A			N/A
9		ReLU		N/A		N/A			N/A
10		Convolutional	36864		9216			0.750000
11		BatchNorm	N/A		N/A			N/A
12		ReLU		N/A		N/A			N/A
13		Convolutional	73728		18432			0.750000
14		BatchNorm	N/A		N/A			N/A
15		ReLU		N/A		N/A			N/A
16		Convolutional	147456		36864			0.750000
17		BatchNorm	N/A		N/A			N/A
18		ReLU		N/A		N/A			N/A
19		Convolutional	147456		36864			0.750000
20		BatchNorm	N/A		N/A			N/A
21		ReLU		N/A		N/A			N/A
22		Convolutional	294912		73728			0.750000
23		BatchNorm	N/A		N/A			N/A
24		ReLU		N/A		N/A			N/A
25		Convolutional	589824		147456			0.750000
26		BatchNorm	N/A		N/A			N/A
27		ReLU		N/A		N/A			N/A
28		Convolutional	589824		147456			0.750000
29		BatchNorm	N